#Tesi

##Download dei dati


###Download cityscapes "gtFine_trainvaltest"

In [ ]:
!wget -P /content/drive/MyDrive/Tesi --keep-session-cookies --save-cookies=cookies.txt --post-data  https://www.cityscapes-dataset.com/login/
!wget -P /content/drive/MyDrive/Tesi --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=1

In [ ]:
%cd /content/drive/MyDrive/Tesi
!pwd
!unzip gtFine_trainvaltest.zip  -d /content/drive/MyDrive/Tesi/gtFine

###Download CityScapes "leftImg8bit_trainvaltest"

In [ ]:
!wget -P /content/drive/MyDrive/Tesi --keep-session-cookies --save-cookies=cookies.txt --post-data  https://www.cityscapes-dataset.com/login/
!wget -P /content/drive/MyDrive/Tesi --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=3

In [ ]:
%cd /content/drive/MyDrive/Tesi
!pwd
!unzip leftImg8bit_trainvaltest.zip  -d /content/drive/MyDrive/Tesi/leftImg8bit

In [ ]:
from __future__ import division
import numpy as np
import keras.backend as K
from tensorflow.keras.layers import Layer, InputSpec
import random
import os.path
import scipy.misc
import shutil
import time
import tensorflow as tf
from glob import glob
import matplotlib.pyplot as plt
import imageio
from collections import namedtuple
from timeit import default_timer as timer
import keras
from keras.models import Model
from keras.layers import Input, Concatenate, Activation, add
from keras.layers import Convolution2D, MaxPooling2D, Convolution2DTranspose, BatchNormalization
import cv2
from keras.models import load_model
import os
from skimage import io
import scipy
from keras.utils.generic_utils import CustomObjectScope
from PIL import Image
from keras.applications import mobilenet
from keras.layers import ReLU#
from keras.layers import DepthwiseConv2D#


##Normalizzazione L2


In [ ]:
class L2Normalization(Layer):
    def __init__(self, gamma_init=20, **kwargs):
        if K.image_data_format() =='channels_last': 
            self.axis = 3
        else:
            self.axis = 1
        self.gamma_init = gamma_init
        super(L2Normalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        gamma = self.gamma_init * np.ones((input_shape[self.axis],))
        self.gamma = K.variable(gamma, name='{}_gamma'.format(self.name))
        self._trainable_weights = [self.gamma]
        super(L2Normalization, self).build(input_shape)

    def call(self, x, mask=None):
        output = K.l2_normalize(x,self.axis)
        return output * self.gamma
        
    def get_config(self):
        config = {
            'gamma_init': self.gamma_init
        }
        base_config = super(L2Normalization, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

##Data processing

In [ ]:
Label = namedtuple('Label', ['name', 'color'])

label_defs = [
    Label('unlabeled',     (0,     0,   0)), Label('dynamic',       (111,  74,   0)),
    Label('ground',        ( 81,   0,  81)), Label('road',          (128,  64, 128)),
    Label('sidewalk',      (244,  35, 232)), Label('parking',       (250, 170, 160)),
    Label('rail track',    (230, 150, 140)), Label('building',      ( 70,  70,  70)),
    Label('wall',          (102, 102, 156)), Label('fence',         (190, 153, 153)),
    Label('guard rail',    (180, 165, 180)), Label('bridge',        (150, 100, 100)),
    Label('tunnel',        (150, 120,  90)), Label('pole',          (153, 153, 153)),
    Label('traffic light', (250, 170,  30)), Label('traffic sign',  (220, 220,   0)),
    Label('vegetation',    (107, 142,  35)), Label('terrain',       (152, 251, 152)),
    Label('sky',           ( 70, 130, 180)), Label('person',        (220,  20,  60)),
    Label('rider',         (255,   0,   0)), Label('car',           (  0,   0, 142)),
    Label('truck',         (  0,   0,  70)), Label('bus',           (  0,  60, 100)),
    Label('caravan',       (  0,   0,  90)), Label('trailer',       (  0,   0, 110)),
    Label('train',         (  0,  80, 100)), Label('motorcycle',    (  0,   0, 230)),
    Label('bicycle',       (119,  11,  32)), Label('pole group',    (153, 153, 153)),]

    
def build_file_list(images_root, labels_root, sample_name):
    image_sample_root = images_root + '/' + sample_name
    image_root_len = len(image_sample_root)
    label_sample_root = labels_root + '/' + sample_name
    image_files = glob(image_sample_root + '/**/*png')
    file_list = []
    for f in image_files:
        f_relative = f[image_root_len:]
        f_dir = os.path.dirname(f_relative)
        f_base = os.path.basename(f_relative)
        f_base_gt = f_base.replace('leftImg8bit', 'gtFine_color')
        f_label = label_sample_root + f_dir + '/' + f_base_gt
        if os.path.exists(f_label):
            file_list.append((f, f_label))
    return file_list


def load_data(data_folder):

    images_root = os.path.join(data_folder, 'leftImg8bit')
    labels_root = os.path.join(data_folder, 'gtFine')

    train_images = build_file_list(images_root, labels_root, 'train')
    valid_images = build_file_list(images_root, labels_root, 'val')
    test_images = build_file_list(images_root, labels_root, 'test')
    num_classes = len(label_defs)
    label_colors = {i: np.array(l.color) for i, l in enumerate(label_defs)}
    
    return train_images, valid_images, test_images, num_classes, label_colors 


def bc_img(img, s=1.0, m=0.0):
    img = img.astype(np.int)
    img = img * s + m
    img[img > 255] = 255
    img[img < 0] = 0
    img = img.astype(np.uint8)
    return img


def gen_batch_function(image_paths, image_shape, test = False):
   
    def get_batches_fn(batch_size):
       
        random.shuffle(image_paths)

        while True:

            for batch_i in range(0, len(image_paths), batch_size):
                image_files = image_paths[batch_i:batch_i+batch_size]

                images = []
                labels = []

                for f in image_files:
                    image_file = f[0]
                    gt_image_file = f[1]
                    image =np.array(Image.fromarray(imageio.imread(image_file)).resize((image_shape[1],image_shape[0])))
                    gt_image =np.array(Image.fromarray(imageio.imread(gt_image_file,as_gray=False, pilmode="RGB")).resize((image_shape[1],image_shape[0])))
                    contrast = random.uniform(0.85, 1.15)  # Contrast augmentation
                    bright = random.randint(-45, 30)  # Brightness augmentation
                    image = bc_img(image, contrast, bright)

                    label_bg = np.zeros([image_shape[0], image_shape[1]], dtype=bool)
                    label_list = []
                    for ldef in label_defs[1:]:
                        
                        x=np.abs(np.float32(gt_image[:,:,0])-ldef.color[0])+np.abs(np.float32(gt_image[:,:,1])-ldef.color[1])+np.abs(np.float32(gt_image[:,:,2])-ldef.color[2])
                        label_current=x<90
                        label_bg |= label_current                   
                        label_list.append(label_current)

                    label_bg = ~label_bg
                    label_all = np.dstack([label_bg, *label_list])

                    label_all = label_all.astype(np.float32)
                    
                    images.append(image)
                    labels.append(label_all)
                    
                if not test:
                    yield np.array(images), np.array(labels)
                else:
                    yield np.array(images)
    return get_batches_fn

##Definizione Rete

In [ ]:
class squeeze_segNet():

    def __init__(self, n_labels, image_shape, weights_path):
        self.n_labels = n_labels
        self.image_shape = image_shape
        self.pretrain_weights_path = weights_path
        pass

    def fire_module(self, x, filters, name="fire"):
        sq_filters, ex1_filters, ex2_filters = filters
        squeeze = Convolution2D(sq_filters, (1, 1), activation='elu', padding='same', name=name + "_squeeze1x1")(x)
        expand1 = Convolution2D(ex1_filters, (1, 1), activation='elu', padding='same', name=name + "_expand1x1")(squeeze)
        expand2 = Convolution2D(ex2_filters, (3, 3), activation='elu', padding='same', name=name + "_expand3x3")(squeeze)
        x = Concatenate(axis=-1, name=name)([expand1, expand2])
        return x

    def squeeze_net(self, x):
        
        x = Convolution2D(64, kernel_size=(3, 3), strides=(2, 2), padding="same", activation="elu", name='conv1')(x)
        x_low1 = x

        x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool1', padding="same")(x)

        x = self.fire_module(x, (16, 64, 64), name="fire2")
        x = self.fire_module(x, (16, 64, 64), name="fire3")

        x_low2 = x
        x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool3', padding="same")(x)

        x = self.fire_module(x, (32, 128, 128), name="fire4")
        x = self.fire_module(x, (32, 128, 128), name="fire5")

        x_low3 = x
        x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool5', padding="same")(x)

        x = self.fire_module(x, (48, 192, 192), name="fire6")
        x = self.fire_module(x, (48, 192, 192), name="fire7")

        x = self.fire_module(x, (64, 256, 256), name="fire8")
        x = self.fire_module(x, (64, 256, 256), name="fire9")

        return x, x_low1, x_low2, x_low3

    def paral_dilat_module(self, x, n_filter=128):
        
        x1 = Convolution2D(64, kernel_size=(3,3), dilation_rate=(1,1), activation='elu', kernel_initializer='he_normal', padding='same', name='dilat1')(x)
        x2 = Convolution2D(64, kernel_size=(3,3), dilation_rate=(3,3), activation='elu', kernel_initializer='he_normal', padding='same', name='dilat2')(x)
        x3 = Convolution2D(64, kernel_size=(3,3), dilation_rate=(5,5), activation='elu', kernel_initializer='he_normal', padding='same', name='dilat3')(x)
        x4 = Convolution2D(64, kernel_size=(3,3), dilation_rate=(7,7), activation='elu', kernel_initializer='he_normal', padding='same', name='dilat4')(x)

        x_sum = add([x1, x2, x3, x4])

        return x_sum
    
    def conv_trans_block(self, x, out_filters, name_trans):

        x = Convolution2DTranspose(64, (1, 1), activation='elu', padding='same', kernel_initializer='he_normal', name=name_trans+'_tran1')(x)
        x = Convolution2DTranspose(64, (3, 3), strides=(2, 2), activation='elu', padding='same', kernel_initializer='he_normal', name=name_trans+'_tran2')(x)
        x = Convolution2DTranspose(out_filters, (1, 1), activation='elu', padding='same', kernel_initializer='he_normal', name=name_trans+'_tran3')(x)

        return x 
    
    def refine_block(self, x, x_low, refine_name):
        x = Convolution2D(64, kernel_size=(3,3), activation='elu', kernel_initializer='he_normal', padding='same', name=refine_name+'_block1')(x)
        
        x = L2Normalization(gamma_init=20, name=refine_name+'l2_norm1')(x)

        x_low = Convolution2D(64, kernel_size=(3,3), activation='elu', kernel_initializer='he_normal', padding='same', name=refine_name+'_block2')(x_low)
        
        x_low = L2Normalization(gamma_init=20, name=refine_name+'l2_norm2')(x_low)

        
        x_sum1 = add([x, x_low])

        return x_sum1

    def conv_transpose(self, x, x_low1, x_low2, x_low3):
        x = self.conv_trans_block(x, 256, name_trans='tran1')
        x = self.refine_block(x, x_low3, refine_name='refine1')
        x = self.conv_trans_block(x, 128, name_trans='tran2')
        x = self.refine_block(x, x_low2, refine_name= 'refine2')
        x = self.conv_trans_block(x, 64, name_trans='tran3')
        x = self.refine_block(x, x_low1, refine_name='refine3')
        x = Convolution2DTranspose(64, (3, 3), strides=(2, 2), activation='elu', padding='same', kernel_initializer='he_normal', name='lasttran')(x)
        x = Convolution2D(self.n_labels, kernel_size=(1,1), activation='softmax', kernel_initializer='he_normal', padding='same', name='lastconv')(x)

        return x



    def init_model(self):
        h, w, d = self.image_shape

        input1 = Input(shape=(h,w,d), name='input')
        output1, x_low1, x_low2, x_low3 = self.squeeze_net(input1)
        
        output_3 = self.paral_dilat_module(output1)
        result = self.conv_transpose(output_3, x_low1, x_low2, x_low3)

        squeeze_seg = Model(inputs=input1, outputs=result, name='squeeze_seg')
        opt = tf.keras.optimizers.RMSprop(learning_rate = 1e-4, rho=0.9, epsilon=1e-08, decay=0.0)
        squeeze_seg.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[iou_metric])

        return squeeze_seg

##Training e Test

In [ ]:
  def iou_metric(y_true, y_pred):
        """
        Indice prestazionale per la segmentazione chiamato Intersection-over-Union o Jaccard index.
        IoU = (|X & Y|)/ (|X| + |Y| - |X & Y|)
        """
        yt = y_true[:,:,:,-1:] > 0.5
        yp = y_pred[:,:,:,-1:] > 0.5
        intersection = K.sum(K.cast_to_floatx(yt & yp), axis=(1,2))
        union = K.sum(K.cast_to_floatx(yt | yp), axis=(1,2))
        iou = intersection / K.maximum(union, 0.01)  # the maximum is used to avoid the divisions with zero
        return iou    


def train(n_labels, batch_size, epochs, train_steps, val_steps, saved_weight, input_shape, pre_weight, train_generator, valid_generator):
    
    os.path.dirname(saved_weight)
    os.makedirs(os.path.dirname(saved_weight),exist_ok=True)
        

    context_Net = squeeze_segNet(n_labels=n_labels, image_shape=input_shape, weights_path = pre_weight)
    model = context_Net.init_model()
    
                                                      
    chk = keras.callbacks.ModelCheckpoint(saved_weight, monitor='val_iou_metric', verbose=0, save_best_only=True, 
                                                                      save_weights_only=True, mode='auto', save_freq='epoch')
    
    model.fit(train_generator(batch_size), steps_per_epoch=train_steps, epochs=epochs, verbose=1, callbacks=chk,
                                                                        validation_data = valid_generator(batch_size), validation_steps=train_steps)
  
def build_pridictor(output_last, label_colors, image_shape):
   
    labels = np.argmax(output_last, axis=-1)
    labels = labels.reshape(image_shape[0], image_shape[1])
    labels_colored = np.zeros((image_shape[0], image_shape[1], 3)) 

    for label, color in label_colors.items():
        labels_colored[labels == label] = color

    return labels_colored

def test(batch_size, test_steps, image_shape, saved_preImage, saved_weight, label_colors, test_generator,n_labels,input_shape, pre_weight):
   
    if not os.path.exists(saved_preImage):
        os.makedirs(saved_preImage)
        #pass

    context_Net = squeeze_segNet(n_labels=n_labels, image_shape=input_shape, weights_path = pre_weight)
    model = context_Net.init_model()
    
    model.load_weights(saved_weight)
    
    predicted = model.predict_generator(test_generator(batch_size), steps=test_steps)
    for index, image in enumerate(predicted):
        saved_images = os.path.join(saved_preImage, str(index)+'.png')
        pred_image = build_pridictor(image, label_colors, image_shape)
        pred_image =np.array(Image.fromarray(np.uint8(pred_image*255)).resize((512,256)))
        cv2.imwrite(saved_images, cv2.cvtColor(pred_image, cv2.COLOR_RGB2BGR))


data_dir = '/content/drive/MyDrive/Triennale/Tesi/Datasets'  # the path of the dataset
pre_weight = None 
saved_weight = os.path.join(data_dir, 'sq_weight.h5')  
saved_preImage = os.path.join(data_dir, 'test') # the path to save the predicted results

image_shape = (128, 256, 3)  # the shape of the input images of the model
target_shape = (128, 256)

batch_size = 5
epochs = 2


train_images, valid_images, test_images, num_classes, label_colors = load_data(data_dir)

train_steps =int(len(train_images)/batch_size) 
val_steps =int(len(valid_images)/batch_size) 
test_steps = int(len(test_images)/batch_size) 


get_train_image = gen_batch_function(train_images, image_shape[0:2])
get_val_image =gen_batch_function(valid_images, image_shape[0:2])
get_test_image =gen_batch_function(test_images, image_shape[0:2], test=True)

                                                                                
train(num_classes, batch_size, epochs, train_steps, val_steps, saved_weight, image_shape, pre_weight, get_train_image, get_val_image)
test(batch_size, test_steps, target_shape, saved_preImage, saved_weight, label_colors, get_test_image,num_classes,image_shape,pre_weight)
  

##Test v 2.0

In [ ]:
def test2(batch_size, test_steps, image_shape, saved_preImage, saved_weight, label_colors, test_generator,n_labels,input_shape, pre_weight):
    
    if not os.path.exists(saved_preImage):
        os.makedirs(saved_preImage)
        #pass

    context_Net = squeeze_segNet(n_labels=n_labels, image_shape=input_shape, weights_path = pre_weight)
    model = context_Net.init_model()
    model.load_weights(saved_weight)
    
    x = model.evaluate(test_generator(batch_size), steps=test_steps,return_dict=True )
    print(x)

    for image,labels in test_generator(batch_size):
      predict=model.predict(image)
      for index in range (len(labels)):
        gt=labels[index]
        out=predict[index]
        saved_images = os.path.join(saved_preImage, str(index)+'_gt.png')
        pred_image = build_pridictor(gt, label_colors, image_shape)
        pred_image =np.array(Image.fromarray(np.uint8(pred_image*255)).resize((512,256)))
        cv2.imwrite(saved_images, cv2.cvtColor(pred_image, cv2.COLOR_RGB2BGR))
        saved_images = os.path.join(saved_preImage, str(index)+'_out.png')
        pred_image = build_pridictor(out, label_colors, image_shape)
        pred_image =np.array(Image.fromarray(np.uint8(pred_image*255)).resize((512,256)))
        cv2.imwrite(saved_images, cv2.cvtColor(pred_image, cv2.COLOR_RGB2BGR))
      break
    

In [ ]:
data_dir = '/content/drive/MyDrive/Triennale/Tesi/Datasets'  # the path of the dataset
pre_weight = None 
saved_weight = os.path.join(data_dir, 'sq_weight.h5')  # the path to save the entire the model
saved_preImage = os.path.join(data_dir, 'test') # the path to save the predicted results

image_shape = (128, 256, 3)  # the shape of the input images of the model
target_shape = (128, 256)

batch_size = 5
epochs = 10


train_images, valid_images, test_images, num_classes, label_colors = load_data(data_dir)

train_steps = 5 
val_steps = 5 
test_steps = 5 
print(train_steps, val_steps, test_steps)

get_train_image = gen_batch_function(train_images, image_shape[0:2])
get_val_image =gen_batch_function(valid_images, image_shape[0:2])
get_test_image =gen_batch_function(test_images, image_shape[0:2], test=True)

                                                            
test2(batch_size, test_steps, target_shape, saved_preImage, saved_weight, label_colors, get_val_image,num_classes,image_shape,pre_weight)
